In [1]:
import argparse
import logging
import db.mariadb as mariadb
import os
import MySQLdb

import imageio
import numpy as np

import keras
from keras.preprocessing import image

from keras.applications.resnet50 import preprocess_input
from requests_futures.sessions import FuturesSession
from PIL.Image import DecompressionBombError

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logging.basicConfig(format='%(asctime)s-%(levelname)s-%(name)s - %(message)s')


model = keras.applications.resnet50.ResNet50(include_top=False, pooling='avg')


Using TensorFlow backend.
2019-03-14 20:26:43,311-WARNING-tensorflow - From /usr/lib/python3.7/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [2]:
# Change according to your database setup (these are the defaults, see .env file)

con = mariadb.get_connection("127.0.0.1", 3308, "image_processing", "user", "user_pw")

files_batch = mariadb.get_files_data(0, 800000, con)

con.close()

# Use every 100th image of the 800000
files_batch = files_batch[::100]
features_train = []
features_test = []

features_train_info = []
features_test_info = []

# Create training and test features
counter = 1
for (img_id, name, path, url) in files_batch:
    try:
        img = image.load_img(path, target_size=(224, 224))
        # print(type(img))
        img_data = image.img_to_array(img)
        # print(type(img_data))
        # print(img_data.shape)
        img_data = np.expand_dims(img_data, axis=0)
        # print(img_data.shape)
        img_data = preprocess_input(img_data)
        # print(img_data.shape)
        res_net_feature = model.predict(img_data)
        # print(res_net_feature.shape)
        res_net_feature = np.array(res_net_feature).flatten()
        # print(res_net_feature.shape)

        if counter % 3 == 0:
            features_test.append(res_net_feature)
            features_test_info.append((img_id, name, path, url))
        else:
            features_train.append(res_net_feature)
            features_train_info.append((img_id, name, path, url))

        counter += 1

    except OSError as e:
        print(e)
    except DecompressionBombError as e:
        print(e)
        print(path)

features_train = np.array(features_train)
features_test = np.array(features_test)

print(features_train.shape)
print(len(features_train_info))

print(features_test.shape)
print(len(features_test_info))

cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/FADatenbankabb0700c/oA_14610.jpg'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/FADatenbankabb0700b/FA-Kae54-22_002921808,02.jpg'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/FADatenbankabb0700d/FA996-12_0002921601,11.jpg'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/FADatenbankabb0700d/Mal1043-01_21751,03.jpg'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/FADatenbankabb0700d/Mal18-07_140891,01.jpg'


/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (131627440 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (96499176 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (129393132 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/FADatenbankabb0575/D-DAI-ROM-56.927_002922407.jpg'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/FADatenbankabb0577/D-DAI-ROM-67.16_56977.jpg'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/FADatenbankabb0577/D-DAI-ROM-71.624_000300141352.jpg'


/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (100186268 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Image size (205393290 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.
/home/shohl/Bilder/idai_cloud_mount/dai-rom-fotothek-2007/Bestand-D-DAI-ROM-2007.5627.JPG


/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (163745498 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (127422594 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0067.JPG'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0175.JPG'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0277.JPG'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0376.JPG'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0594.JPG'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0694.JPG'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0794.JPG'
(5323, 2048)
5323
(2661, 2048)
2661


In [3]:
# We need to scale the features to values between 0 and 1, because the sigmoid layer (last autoencoder layer) 
# produces values in that range. Without scaling the original values down, the autoencoder won't be able to 
# reproduce input values > 1.

print(np.max(features_train))
print(np.max(features_test))

if(np.max(features_train) > np.max(features_test)):
    features_train_scaled = features_train / np.max(features_train)
    features_test_scaled = features_test / np.max(features_train)
else:
    features_train_scaled = features_train / np.max(features_test)
    features_test_scaled = features_test / np.max(features_test)

print(np.max(features_train_scaled))
print(np.max(features_test_scaled))

29.489676
21.073988
1.0
0.71462256


In [4]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=10)

result = neigh.fit(features_train_scaled)

neighbours = result.kneighbors()

In [5]:
print(len(neighbours))
print(len(neighbours[0]))
print(neighbours[0])
print(len(neighbours[1]))
print(neighbours[1])

2
5323
[[0.8869949  0.95451961 0.96011991 ... 0.9791909  0.98546634 0.9955557 ]
 [1.26883223 1.30483152 1.3814718  ... 1.44745823 1.44969341 1.45190205]
 [1.1523444  1.19131711 1.20262143 ... 1.2408876  1.2468743  1.2603429 ]
 ...
 [1.03671313 1.03765333 1.06060254 ... 1.08386614 1.08407201 1.08443034]
 [1.06930514 1.07236736 1.08549986 ... 1.11641227 1.11959069 1.12072677]
 [0.74386297 0.77322731 0.79062213 ... 0.83160336 0.83692799 0.84112598]]
5323
[[1821 2854 2761 ... 3762 2723 2719]
 [ 721 2822 3091 ...  208 4134 1022]
 [ 184  204 1096 ... 1089 2822 3232]
 ...
 [4274 2145 4962 ... 1395 2131 3367]
 [5193 1383 4025 ... 1060 4137 1394]
 [5192 5314  160 ... 4712 5181 1056]]


In [7]:
# Initialize and run autoencoder

from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import Adam, Adadelta
from keras import regularizers
import keras.backend as K

def euclidean_distance_loss(y_true, y_pred):
    """
    Euclidean distance loss
    https://en.wikipedia.org/wiki/Euclidean_distance
    :param y_true: TensorFlow/Theano tensor
    :param y_pred: TensorFlow/Theano tensor of the same shape as y_true
    :return: float
    """
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

# Stop if there has been no improvement for 100 epochs
es = keras.callbacks.EarlyStopping(verbose=1, patience=100)
# Save best model while training
mc = keras.callbacks.ModelCheckpoint('best_model.h5')

callbacks_list = [es, mc]

m = Sequential()

m.add(Dense(512, activation='elu', input_shape=(features_train_scaled.shape[1],)))
m.add(Dense(32, activation='linear', name="bottleneck"))
m.add(Dense(512, activation='elu'))
m.add(Dense(features_train.shape[1], activation='sigmoid'))

m.compile(loss=euclidean_distance_loss, optimizer=Adadelta())

history = m.fit(features_train_scaled, features_train_scaled, batch_size=128, epochs=100000, verbose=1,
                validation_data=(features_test_scaled, features_test_scaled), callbacks=callbacks_list)


Train on 5323 samples, validate on 2661 samples
Epoch 1/100000
5323/5323 [==============================] - 4s 781us/step - loss: 5.6196 - val_loss: 1.1665
Epoch 2/100000
5323/5323 [==============================] - 3s 625us/step - loss: 1.1611 - val_loss: 1.1474
Epoch 3/100000
5323/5323 [==============================] - 3s 623us/step - loss: 1.1460 - val_loss: 1.1341
Epoch 4/100000
5323/5323 [==============================] - 3s 627us/step - loss: 1.1328 - val_loss: 1.1305
Epoch 5/100000
5323/5323 [==============================] - 3s 628us/step - loss: 1.1253 - val_loss: 1.1204
Epoch 6/100000
5323/5323 [==============================] - 3s 623us/step - loss: 1.1161 - val_loss: 1.1163
Epoch 7/100000
5323/5323 [==============================] - 3s 630us/step - loss: 1.1124 - val_loss: 1.1059
Epoch 8/100000
5323/5323 [==============================] - 3s 640us/step - loss: 1.1069 - val_loss: 1.1078
Epoch 9/100000
5323/5323 [==============================] - 3s 634us/step - loss: 1.1031

Epoch 76/100000
5323/5323 [==============================] - 3s 623us/step - loss: 0.8181 - val_loss: 0.8207
Epoch 77/100000
5323/5323 [==============================] - 3s 623us/step - loss: 0.8141 - val_loss: 0.8120
Epoch 78/100000
5323/5323 [==============================] - 3s 624us/step - loss: 0.8134 - val_loss: 0.8109
Epoch 79/100000
5323/5323 [==============================] - 3s 622us/step - loss: 0.8112 - val_loss: 0.8203
Epoch 80/100000
5323/5323 [==============================] - 3s 623us/step - loss: 0.8111 - val_loss: 0.8118
Epoch 81/100000
5323/5323 [==============================] - 3s 616us/step - loss: 0.8073 - val_loss: 0.8073
Epoch 82/100000
5323/5323 [==============================] - 3s 616us/step - loss: 0.8041 - val_loss: 0.8027
Epoch 83/100000
5323/5323 [==============================] - 3s 620us/step - loss: 0.8028 - val_loss: 0.8034
Epoch 84/100000
5323/5323 [==============================] - 3s 619us/step - loss: 0.8008 - val_loss: 0.8029
Epoch 85/100000
532

5323/5323 [==============================] - 3s 637us/step - loss: 0.7001 - val_loss: 0.7114
Epoch 151/100000
5323/5323 [==============================] - 3s 621us/step - loss: 0.6991 - val_loss: 0.7121
Epoch 152/100000
5323/5323 [==============================] - 3s 624us/step - loss: 0.6972 - val_loss: 0.7101
Epoch 153/100000
5323/5323 [==============================] - 3s 619us/step - loss: 0.6970 - val_loss: 0.7077
Epoch 154/100000
5323/5323 [==============================] - 3s 626us/step - loss: 0.6959 - val_loss: 0.7144
Epoch 155/100000
5323/5323 [==============================] - 3s 617us/step - loss: 0.6955 - val_loss: 0.7019
Epoch 156/100000
5323/5323 [==============================] - 3s 619us/step - loss: 0.6931 - val_loss: 0.7039
Epoch 157/100000
5323/5323 [==============================] - 3s 620us/step - loss: 0.6915 - val_loss: 0.6982
Epoch 158/100000
5323/5323 [==============================] - 3s 622us/step - loss: 0.6903 - val_loss: 0.7006
Epoch 159/100000
5323/5323 

5323/5323 [==============================] - 3s 609us/step - loss: 0.6371 - val_loss: 0.6586
Epoch 225/100000
5323/5323 [==============================] - 3s 609us/step - loss: 0.6354 - val_loss: 0.6569
Epoch 226/100000
5323/5323 [==============================] - 3s 609us/step - loss: 0.6356 - val_loss: 0.6541
Epoch 227/100000
5323/5323 [==============================] - 3s 611us/step - loss: 0.6348 - val_loss: 0.6569
Epoch 228/100000
5323/5323 [==============================] - 3s 623us/step - loss: 0.6347 - val_loss: 0.6516
Epoch 229/100000
5323/5323 [==============================] - 3s 611us/step - loss: 0.6329 - val_loss: 0.6548
Epoch 230/100000
5323/5323 [==============================] - 3s 613us/step - loss: 0.6334 - val_loss: 0.6552
Epoch 231/100000
5323/5323 [==============================] - 3s 613us/step - loss: 0.6322 - val_loss: 0.6509
Epoch 232/100000
5323/5323 [==============================] - 3s 621us/step - loss: 0.6306 - val_loss: 0.6592
Epoch 233/100000
5323/5323 

5323/5323 [==============================] - 3s 615us/step - loss: 0.5973 - val_loss: 0.6317
Epoch 299/100000
5323/5323 [==============================] - 3s 618us/step - loss: 0.5980 - val_loss: 0.6317
Epoch 300/100000
5323/5323 [==============================] - 3s 610us/step - loss: 0.5989 - val_loss: 0.6285
Epoch 301/100000
5323/5323 [==============================] - 3s 609us/step - loss: 0.5970 - val_loss: 0.6324
Epoch 302/100000
5323/5323 [==============================] - 3s 611us/step - loss: 0.5966 - val_loss: 0.6354
Epoch 303/100000
5323/5323 [==============================] - 3s 615us/step - loss: 0.5970 - val_loss: 0.6334
Epoch 304/100000
5323/5323 [==============================] - 3s 612us/step - loss: 0.5960 - val_loss: 0.6298
Epoch 305/100000
5323/5323 [==============================] - 3s 614us/step - loss: 0.5967 - val_loss: 0.6313
Epoch 306/100000
5323/5323 [==============================] - 3s 612us/step - loss: 0.5946 - val_loss: 0.6315
Epoch 307/100000
5323/5323 

5323/5323 [==============================] - 3s 612us/step - loss: 0.5754 - val_loss: 0.6276
Epoch 373/100000
5323/5323 [==============================] - 3s 610us/step - loss: 0.5748 - val_loss: 0.6246
Epoch 374/100000
5323/5323 [==============================] - 3s 615us/step - loss: 0.5741 - val_loss: 0.6248
Epoch 375/100000
5323/5323 [==============================] - 3s 609us/step - loss: 0.5737 - val_loss: 0.6228
Epoch 376/100000
5323/5323 [==============================] - 3s 613us/step - loss: 0.5735 - val_loss: 0.6234
Epoch 377/100000
5323/5323 [==============================] - 3s 610us/step - loss: 0.5726 - val_loss: 0.6245
Epoch 378/100000
5323/5323 [==============================] - 3s 610us/step - loss: 0.5751 - val_loss: 0.6270
Epoch 379/100000
5323/5323 [==============================] - 3s 613us/step - loss: 0.5739 - val_loss: 0.6232
Epoch 380/100000
5323/5323 [==============================] - 3s 614us/step - loss: 0.5725 - val_loss: 0.6224
Epoch 381/100000
5323/5323 

5323/5323 [==============================] - 3s 610us/step - loss: 0.5579 - val_loss: 0.6218
Epoch 447/100000
5323/5323 [==============================] - 3s 612us/step - loss: 0.5575 - val_loss: 0.6240
Epoch 448/100000
5323/5323 [==============================] - 3s 612us/step - loss: 0.5564 - val_loss: 0.6202
Epoch 449/100000
5323/5323 [==============================] - 3s 611us/step - loss: 0.5577 - val_loss: 0.6175
Epoch 450/100000
5323/5323 [==============================] - 3s 612us/step - loss: 0.5562 - val_loss: 0.6193
Epoch 451/100000
5323/5323 [==============================] - 3s 614us/step - loss: 0.5572 - val_loss: 0.6182
Epoch 452/100000
5323/5323 [==============================] - 3s 612us/step - loss: 0.5557 - val_loss: 0.6199
Epoch 453/100000
5323/5323 [==============================] - 3s 611us/step - loss: 0.5560 - val_loss: 0.6209
Epoch 454/100000
5323/5323 [==============================] - 3s 612us/step - loss: 0.5562 - val_loss: 0.6229
Epoch 455/100000
5323/5323 

5323/5323 [==============================] - 3s 612us/step - loss: 0.5444 - val_loss: 0.6207
Epoch 521/100000
5323/5323 [==============================] - 3s 612us/step - loss: 0.5452 - val_loss: 0.6185
Epoch 522/100000
5323/5323 [==============================] - 3s 613us/step - loss: 0.5448 - val_loss: 0.6186
Epoch 523/100000
5323/5323 [==============================] - 3s 610us/step - loss: 0.5450 - val_loss: 0.6189
Epoch 524/100000
5323/5323 [==============================] - 3s 612us/step - loss: 0.5439 - val_loss: 0.6175
Epoch 525/100000
5323/5323 [==============================] - 3s 611us/step - loss: 0.5432 - val_loss: 0.6192
Epoch 526/100000
5323/5323 [==============================] - 3s 613us/step - loss: 0.5444 - val_loss: 0.6268
Epoch 527/100000
5323/5323 [==============================] - 3s 616us/step - loss: 0.5434 - val_loss: 0.6221
Epoch 528/100000
5323/5323 [==============================] - 3s 630us/step - loss: 0.5438 - val_loss: 0.6187
Epoch 529/100000
5323/5323 

5323/5323 [==============================] - 3s 611us/step - loss: 0.5325 - val_loss: 0.6203
Epoch 595/100000
5323/5323 [==============================] - 3s 614us/step - loss: 0.5332 - val_loss: 0.6169
Epoch 596/100000
5323/5323 [==============================] - 3s 613us/step - loss: 0.5335 - val_loss: 0.6182
Epoch 597/100000
5323/5323 [==============================] - 3s 608us/step - loss: 0.5341 - val_loss: 0.6179
Epoch 598/100000
5323/5323 [==============================] - 3s 612us/step - loss: 0.5338 - val_loss: 0.6202
Epoch 599/100000
5323/5323 [==============================] - 3s 618us/step - loss: 0.5333 - val_loss: 0.6195
Epoch 600/100000
5323/5323 [==============================] - 3s 612us/step - loss: 0.5335 - val_loss: 0.6164
Epoch 601/100000
5323/5323 [==============================] - 3s 615us/step - loss: 0.5335 - val_loss: 0.6179
Epoch 602/100000
5323/5323 [==============================] - 3s 613us/step - loss: 0.5321 - val_loss: 0.6200
Epoch 603/100000
5323/5323 

In [9]:
from keras.models import load_model
best_model = load_model('best_model.h5', custom_objects={'euclidean_distance_loss': euclidean_distance_loss})
encoder = Model(best_model.input, best_model.get_layer('bottleneck').output)

enc = encoder.predict(features_train_scaled) # returns the encoded values (32 floats instead of 2048)


In [10]:
result_enc = neigh.fit(enc)
neighbours_enc = result_enc.kneighbors()

In [11]:
enc_r = best_model.predict(features_train_scaled)      # reconstruction, 2048 -> 32 -> 2048

In [12]:
result_enc_r = neigh.fit(enc_r)
neighbours_enc_r = result_enc.kneighbors()

In [13]:
np.min(features_train_scaled)

0.0

In [14]:
np.min(features_test_scaled)

0.0

In [15]:
np.max(features_train_scaled)

1.0

In [16]:
np.max(features_test_scaled)

0.71462256

In [17]:
print(neighbours[1][0])
print(neighbours_enc_r[1][0])
print(neighbours_enc[1][0])

[1821 2854 2761 3123 3441 1823 3599 3762 2723 2719]
[1821 2854 4093 4518 3115 4012 3762 1499 5149 3123]
[4093 4518 1821 3762 2723 2642 2854 4012 3225 4018]


In [18]:
print(neighbours[1][1])
print(neighbours_enc_r[1][1])
print(neighbours_enc[1][1])

[ 721 2822 3091 4785 4879 2877 1107  208 4134 1022]
[ 721 2822 3091  150 4785 1365 4879  208 1088 1107]
[ 721 2822  150  187 1107  173 1022  185 1092  177]


In [19]:
print(neighbours[1][2])
print(neighbours_enc_r[1][2])
print(neighbours_enc[1][2])

[ 184  204 1096  225 1090 1088 1085 1089 2822 3232]
[ 204  184 1090 1088 1107  225 1089 1096 3232 1085]
[ 204 1090 1088  184  225 1107 1089 1092  173 2495]


In [20]:
enc.shape

(5323, 32)

In [21]:
features_train_scaled.shape

(5323, 2048)

In [22]:
encoder.save('encoder.h5')